22206207 Giacomo Govoni 

# Machine Learning with Python Assignment 1

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from fractions import Fraction
from IPython.display import display, Markdown, Latex
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, CategoricalNB
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score
import category_encoders as ce

In [2]:
breast_cancer = pd.read_csv('breast-cancer.csv')
breast_cancer

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


## TASK 1

In [3]:
# I set the number of trials and the first values for task 1
trials = 200
X_q1 = breast_cancer
y_q1 = X_q1.pop('Class').values

In [4]:
# OneHot Encoder

ohe_q1 = OneHotEncoder(sparse=False)
X_OH_q1 = ohe_q1.fit_transform(breast_cancer)

OHEaccur_q1 = []
bnb_q1 = BernoulliNB()

for i in range(trials):
    X_train, X_test, y_train, y_test = train_test_split(X_OH_q1, y_q1, test_size=0.5, random_state=None)
    y_pred = bnb_q1.fit(X_train, y_train).predict(X_test)
    OHEaccur_q1.append(accuracy_score(y_test, y_pred))

In [5]:
display(Markdown(f'Before splitting with BERNOULLI NAIVE BAYES classifier the accuracy of using ONEHOT ENCODER encoding over `{trials}` trials is {(np.mean(OHEaccur_q1))*100:.2f}%'))

Before splitting with BERNOULLI NAIVE BAYES classifier the accuracy of using ONEHOT ENCODER encoding over `200` trials is 72.09%

In [6]:
# Ordinal Encoder

ord_encoder_q1 = OrdinalEncoder()
X_OE_q1 = ord_encoder_q1.fit_transform(breast_cancer)

catNB_q1 = CategoricalNB() 
OEaccur_q1 = []

for i in range(trials):
    X_train, X_test, y_train, y_test = train_test_split(X_OE_q1, y_q1, test_size=0.5, random_state=None)
    try:
        OE_y_pred = catNB_q1.fit(X_train, y_train).predict(X_test)
        OEaccur_q1.append(accuracy_score(y_test, OE_y_pred))
    except:
        continue    

In [7]:
display(Markdown(f'Before splitting with CATEGORICAL NAIVE BAYES classifier the accuracy of using ORDINAL ENCODER encoding over `{trials}` trials is {(np.mean(OEaccur_q1))*100:.2f}%'))

Before splitting with CATEGORICAL NAIVE BAYES classifier the accuracy of using ORDINAL ENCODER encoding over `200` trials is 72.06%

## TASK 2

In [8]:
breast_cancer = pd.read_csv('breast-cancer.csv')

breast_cancer.loc[breast_cancer["Class"] == "no-recurrence-events", "Class"] = 1.0
breast_cancer.loc[breast_cancer["Class"] == "recurrence-events", "Class"] = 0.0

y_q2 = list(breast_cancer.pop('Class').values)
X_q2 = breast_cancer
cbe = ce.cat_boost.CatBoostEncoder(cols = X_q2.columns)

In [ ]:
gnb_q2 = GaussianNB()
accur_q2 = []

for i in range(trials):
    cbe = ce.cat_boost.CatBoostEncoder(cols = X_q2.columns)
    X2_cbe = cbe.fit_transform(X_q2,y_q2)
    X_train, X_test, y_train, y_test = train_test_split(X2_cbe, y_q2, test_size=0.5)
    y_dash = gnb_q2.fit(X_train, y_train).predict(X_test)
    accur_q2.append(accuracy_score(y_test, y_dash))

In [ ]:
display(Markdown(f'Before splitting with GAUSSIAN NAIVE BAYES classifier the accuracy of using CATBOOST ENCODER encoding over `{trials}` trials is {(np.mean(accur_q2))*100:.2f}%'))

## TASK 3

### Now encoding after splitting 

In [ ]:
# To store in an array the unique values in the date column of a particular dataframe

def guv(df):
    coll = list(df.columns)
    categ = []
    for col in coll:
        categ.append(list((df[col].unique())))
    
    return categ

In [ ]:
# OneHotEncoder

OHEaccur_q3 = []
bnb_q3 = BernoulliNB()
X_q3 = breast_cancer.astype(str)
y_q3 = y_q1

for i in range(trials):
    X_train, X_test, y_train, y_test = train_test_split(X_q3, y_q3, test_size=0.5, random_state=None)
    ohe_q3 = OneHotEncoder(sparse=False, categories=guv(X_q3))
    OH_BC_q3_train = ohe_q3.fit_transform(X_train)
    OH_BC_q3_test = ohe_q3.fit_transform(X_test)
    X_OH_q3 = OH_BC_q3_train
    y_pred = bnb_q3.fit(OH_BC_q3_train, y_train).predict(OH_BC_q3_test)
    OHEaccur_q3.append(accuracy_score(y_test, y_pred))

In [ ]:
display(Markdown(f'After splitting with BERNOULLI NAIVE BAYES classifier the accuracy of using ONEHOT ENCODER encoding over `{trials}` trials is {(np.mean(OHEaccur_q3))*100:.2f}%'))

In [ ]:
# OrdinalEncoder

OEaccur_q3 = []
catNB = CategoricalNB(fit_prior=True, alpha=0.0001)
X_q3_2 = breast_cancer.astype(str)


for i in range(trials):
    y_q3 = y_q1
    X_train, X_test, y_train, y_test = train_test_split(X_q3_2, y_q3, test_size=0.5, random_state=None)

    oe_q3_train = OrdinalEncoder(categories=guv(X_q3))

    OE_BC_q3_train = oe_q3_train.fit_transform(X_train)
    
    OE_BC_q3_test = oe_q3_train.transform(X_test)
    
    X_OE_q3 = OE_BC_q3_train
    
    try:
        y_pred = catNB.fit(OE_BC_q3_train, y_train).predict(OE_BC_q3_test)
        OEaccur_q3.append(accuracy_score(y_test, y_pred))
    except:
        continue

In [ ]:
display(Markdown(f'After splitting with CATEGORICAL NAIVE BAYES classifier the accuracy of using ORDINAL ENCODER encoding over `{trials}` trials is {(np.mean(OEaccur_q3))*100:.2f}%'))

In [ ]:
# CatBoost Encoder

gnb_q3 = GaussianNB()
accur_q3_catb = []
for i in range(trials):
    X_train, X_test, y_train, y_test = train_test_split(X_q2, y_q2, test_size=0.5)
    cbe = ce.cat_boost.CatBoostEncoder(cols = X_q2.columns)
    X2_cbe_train = cbe.fit_transform(X_train,y_train)
    X2_cbe_test = cbe.fit_transform(X_test,y_test)
    y_dash = gnb_q3.fit(X2_cbe_train, y_train).predict(X2_cbe_test)
    accur_q3_catb.append(accuracy_score(y_test, y_dash))

In [ ]:
display(Markdown(f'After splitting with GAUSSIAN NAIVE BAYES classifier the accuracy of using CATBOOST encoding over `{trials}` trials is {(np.mean(accur_q3_catb))*100:.2f}%'))

## TASK 4


In [ ]:
x_1 = [np.mean(OHEaccur_q1), np.mean(OEaccur_q1)]
x_2 = [np.mean(OHEaccur_q3), np.mean(OEaccur_q3)]
bar_1 = ['OneHot', 'Ordinal']
bar_2 = ['OneHot','Ordinal']

fig, ax = plt.subplots(figsize =(14, 10))
encoders = ['OneHot','Ordial','CatBoost']

before = [np.mean(OHEaccur_q1), np.mean(OEaccur_q1), np.mean(accur_q2)]
after = [np.mean(OHEaccur_q3), np.mean(OEaccur_q3), np.mean(accur_q3_catb)]

x_axis = np.arange(len(encoders))
plt.bar(x_axis -0.2, before, width=0.4, label = 'Before')
plt.bar(x_axis +0.2, after, width=0.4, label = 'After')


plt.xticks(x_axis, encoders)
for bars in ax.containers:
    ax.bar_label(bars)
    
plt.ylabel('Accuracy')
plt.title("Encoding Before/After Train-Test Split")
plt.legend()

# Conclusions
In particular, the ONEHOT and ORDINAL encoders do not have much difference between before and after division this is because they encode data independently of each data point.

Whereas the CAT BOOST encoder, on the other hand, has a different operation, specifically it encodes the data based on the frequency of the features. 
From the graph you can see that initially there is a higher accuracy because the encoder has many more data points than after the split.
